In [2]:
%run create_tables.py

INFO: Table Creation Successful!


In [3]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *
import json
from dotenv import dotenv_values

In [4]:
config = dict(dotenv_values(".env"))
conn = psycopg2.connect(**config)
curr = conn.cursor()

In [5]:
log_json_wildcard = os.path.join('.', 'data', 'log_data', '**', '*.json')
song_json_wildcard = os.path.join('.', 'data', 'song_data', '**', '*.json')

In [6]:
log_files = glob.glob(log_json_wildcard, recursive=True)
song_files = glob.glob(song_json_wildcard, recursive=True)


In [7]:
def return_json(fn):
    with open(fn) as f:
        data_list = [json.loads(line) for line in f]
    
    return data_list

In [8]:
flat_song_file = [item for f in song_files for item in return_json(f)]
flat_log_file = [item for f in log_files for item in return_json(f)]

In [9]:
all_songs = pd.DataFrame(flat_song_file)
all_logs = pd.DataFrame(flat_log_file)

In [10]:
all_songs

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,NaN,NaN,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0
1,1,ARMJAGH1187FB546F3,35.14968,-90.04892,"Memphis, TN",The Box Tops,SOCIWDW12A8C13D406,Soul Deep,148.03546,1969
2,1,ARKRRTF1187B9984DA,NaN,NaN,,Sonora Santanera,SOXVLOJ12AB0189215,Amor De Cabaret,177.47546,0
3,1,AR7G5I41187FB4CE6C,NaN,NaN,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982
4,1,ARXR32B1187FB57099,NaN,NaN,,Gob,SOFSOCN12A8C143F5D,Face the Ashes,209.60608,2007
...,...,...,...,...,...,...,...,...,...,...
66,1,AR8IEZO1187B99055E,NaN,NaN,,Marc Shaiman,SOINLJW12A8C13314C,City Slickers,149.86404,2008
67,1,AR558FS1187FB45658,NaN,NaN,,40 Grit,SOGDBUF12A8C140FAA,Intro,75.67628,2003
68,1,ARVBRGZ1187FB4675A,NaN,NaN,,Gwen Stefani,SORRZGD12A6310DBC3,Harajuku Girls,290.55955,2004
69,1,ARWB3G61187FB49404,NaN,NaN,"Hamilton, Ohio",Steve Morse,SODAUVL12A8C13D184,Prognosis,363.85914,2000


In [11]:
next_song_logs = all_logs.loc[all_logs['page'] == 'NextSong'].copy()

In [12]:
next_song_logs.loc[:,'ts_dt'] = next_song_logs['ts'].apply(
    pd.to_datetime, unit='ms', origin='unix'
)

In [13]:
time_df = pd.DataFrame([{
    'start_time': v,
    'hour': v.hour,
    'day': v.day,
    'week': v.week,
    'month': v.month,
    'year': v.year
} for _, v in next_song_logs['ts_dt'].items()])

In [14]:
time_df.drop_duplicates(subset=['start_time'], inplace=True)

In [15]:
from io import StringIO


buffer = StringIO()
time_df.to_csv(buffer, header=False, index=False, sep="\t")
buffer.seek(0)

curr.copy_from(buffer, 'time', sep="\t", columns=tuple(time_df.columns))

In [16]:
conn.commit()

In [17]:
curr.execute("SELECT * FROM time")
curr.fetchall()

[(datetime.datetime(2018, 11, 1, 21, 1, 46, 796000), 21, 1, 44, 11, 2018),
 (datetime.datetime(2018, 11, 1, 21, 5, 52, 796000), 21, 1, 44, 11, 2018),
 (datetime.datetime(2018, 11, 1, 21, 8, 16, 796000), 21, 1, 44, 11, 2018),
 (datetime.datetime(2018, 11, 1, 21, 11, 13, 796000), 21, 1, 44, 11, 2018),
 (datetime.datetime(2018, 11, 1, 21, 17, 33, 796000), 21, 1, 44, 11, 2018),
 (datetime.datetime(2018, 11, 1, 21, 24, 53, 796000), 21, 1, 44, 11, 2018),
 (datetime.datetime(2018, 11, 1, 21, 28, 54, 796000), 21, 1, 44, 11, 2018),
 (datetime.datetime(2018, 11, 1, 21, 42, 0, 796000), 21, 1, 44, 11, 2018),
 (datetime.datetime(2018, 11, 1, 21, 52, 5, 796000), 21, 1, 44, 11, 2018),
 (datetime.datetime(2018, 11, 1, 21, 55, 25, 796000), 21, 1, 44, 11, 2018),
 (datetime.datetime(2018, 11, 1, 22, 23, 14, 796000), 22, 1, 44, 11, 2018),
 (datetime.datetime(2018, 11, 2, 1, 25, 34, 796000), 1, 2, 44, 11, 2018),
 (datetime.datetime(2018, 11, 2, 1, 30, 41, 796000), 1, 2, 44, 11, 2018),
 (datetime.datetime(2

In [43]:
user_df = next_song_logs[['userId', 'firstName', 'lastName', 'gender', 'level', 'ts_dt']]
user_df = user_df.rename(columns={
    'userId': 'user_id',
    'firstName': 'first_name',
    'lastName': 'last_name'
})

In [45]:
user_df.sort_values('ts_dt')

,user_id,first_name,last_name,gender,level,ts_dt
2,8,Kaylee,Summers,F,free,2018-11-01 21:01:46.796
4,8,Kaylee,Summers,F,free,2018-11-01 21:05:52.796
5,8,Kaylee,Summers,F,free,2018-11-01 21:08:16.796
6,8,Kaylee,Summers,F,free,2018-11-01 21:11:13.796
7,8,Kaylee,Summers,F,free,2018-11-01 21:17:33.796
...,...,...,...,...,...,...
8050,16,Rylan,George,M,paid,2018-11-30 18:40:05.796
8051,16,Rylan,George,M,paid,2018-11-30 18:44:36.796
8052,16,Rylan,George,M,paid,2018-11-30 18:47:58.796
8053,16,Rylan,George,M,paid,2018-11-30 18:51:24.796


In [18]:
user_df.sort_values('ts_dt', inplace=True)

user_df.drop_duplicates('user_id', keep="last", inplace=True)

user_df.drop('ts_dt', axis=1, inplace=True)

In [19]:
conn.rollback()

In [20]:
buffer = StringIO()

user_df.to_csv(buffer, header=False, index=False, sep="\t")
buffer.seek(0)

curr.copy_from(buffer, 'users', sep="\t", columns=tuple(user_df.columns))

In [21]:
conn.commit()

In [22]:
curr.execute("SELECT * FROM users")
curr.fetchall()

[(3, 'Isaac', 'Valdez', 'M', 'free'),
 (84, 'Shakira', 'Hunt', 'F', 'free'),
 (20, 'Aiden', 'Ramirez', 'M', 'paid'),
 (27, 'Carlos', 'Carter', 'M', 'free'),
 (59, 'Lily', 'Cooper', 'F', 'free'),
 (99, 'Ann', 'Banks', 'F', 'free'),
 (34, 'Evelin', 'Ayala', 'F', 'free'),
 (62, 'Connar', 'Moreno', 'M', 'free'),
 (2, 'Jizelle', 'Benjamin', 'F', 'free'),
 (7, 'Adelyn', 'Jordan', 'F', 'free'),
 (45, 'Dominick', 'Norris', 'M', 'free'),
 (70, 'Jaleah', 'Hayes', 'F', 'paid'),
 (71, 'Ayleen', 'Wise', 'F', 'free'),
 (51, 'Maia', 'Burke', 'F', 'free'),
 (25, 'Jayden', 'Graves', 'M', 'paid'),
 (42, 'Harper', 'Barrett', 'M', 'paid'),
 (41, 'Brayden', 'Clark', 'M', 'free'),
 (35, 'Molly', 'Taylor', 'F', 'free'),
 (81, 'Sienna', 'Colon', 'F', 'free'),
 (18, 'Jacob', 'Rogers', 'M', 'free'),
 (64, 'Hannah', 'Calhoun', 'F', 'free'),
 (65, 'Amiya', 'Davidson', 'F', 'paid'),
 (56, 'Cienna', 'Freeman', 'F', 'free'),
 (94, 'Noah', 'Chavez', 'M', 'free'),
 (90, 'Andrea', 'Butler', 'F', 'free'),
 (47, 'Kimber'

In [27]:
unique_artist_id = artists_df['artist_id'].value_counts()

In [42]:
artists_df.loc[artists_df['artist_id'].isin(unique_artist_id[~unique_artist_id.where(unique_artist_id > 1).isna()].index.tolist())]

,artist_id,name,location,latitude,longitude
0,ARD7TVE1187B99BFB1,Casual,California - LA,NaN,NaN
9,ARNTLGG11E2835DDB9,Clp,,NaN,NaN
26,ARNTLGG11E2835DDB9,Clp,,NaN,NaN
36,ARD7TVE1187B99BFB1,Casual,California - LA,NaN,NaN


In [23]:
artists_df = all_songs[
    ['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']
]

artists_df = artists_df.rename(columns={
    'artist_name': 'name',
    'artist_location': 'location',
    'artist_latitude': 'latitude',
    'artist_longitude': 'longitude',
})

In [ ]:

artists_df = artists_df.drop_duplicates(subset='artist_id')

In [23]:
buffer = StringIO()

artists_df.to_csv(
    buffer,
    header=False,
    index=False,
    sep="\t"
)
buffer.seek(0)

curr.copy_from(buffer, 'artists', sep="\t", columns=tuple(artists_df.columns), null="")

In [24]:
curr.execute("SELECT * FROM artists")
curr.fetchall()

[('ARD7TVE1187B99BFB1', 'Casual', 'California - LA', None, None),
 ('ARMJAGH1187FB546F3',
  'The Box Tops',
  'Memphis, TN',
  Decimal('35.14968'),
  Decimal('-90.04892')),
 ('ARKRRTF1187B9984DA', 'Sonora Santanera', None, None, None),
 ('AR7G5I41187FB4CE6C', 'Adam Ant', 'London, England', None, None),
 ('ARXR32B1187FB57099', 'Gob', None, None, None),
 ('ARKFYS91187B98E58F', 'Jeff And Sheri Easter', None, None, None),
 ('ARD0S291187B9B7BF5', 'Rated R', 'Ohio', None, None),
 ('AR10USD1187B99F3F1',
  'Tweeterfriendly Music',
  'Burlington, Ontario, Canada',
  None,
  None),
 ('AR8ZCNI1187B9A069B', 'Planet P Project', None, None, None),
 ('ARNTLGG11E2835DDB9', 'Clp', None, None, None),
 ('ARGSJW91187B9B1D6B',
  'JennyAnyKind',
  'North Carolina',
  Decimal('35.21962'),
  Decimal('-80.01955')),
 ('ARC43071187B990240', 'Wayne Watson', 'Wisner, LA', None, None),
 ('ARL7K851187B99ACD2', 'Andy Andy', None, None, None),
 ('ARHHO3O1187B989413', 'Bob Azzam', None, None, None),
 ('ARIK43K1187B9AE5

In [25]:
conn.commit()

In [26]:
songs_df = all_songs[
    ['song_id', 'title', 'artist_id', 'year', 'duration']
]

In [27]:
buffer = StringIO()

songs_df.to_csv(buffer, header=False, index=False, sep="\t")
buffer.seek(0)

curr.copy_from(buffer, 'songs', sep="\t", columns=tuple(songs_df.columns))

In [28]:
curr.execute("SELECT * FROM songs")
curr.fetchall()

[('SOMZWCG12A8C13C480',
  "I Didn't Mean To",
  'ARD7TVE1187B99BFB1',
  0,
  218.93179),
 ('SOCIWDW12A8C13D406', 'Soul Deep', 'ARMJAGH1187FB546F3', 1969, 148.03546),
 ('SOXVLOJ12AB0189215', 'Amor De Cabaret', 'ARKRRTF1187B9984DA', 0, 177.47546),
 ('SONHOTT12A8C13493C',
  'Something Girls',
  'AR7G5I41187FB4CE6C',
  1982,
  233.40363),
 ('SOFSOCN12A8C143F5D',
  'Face the Ashes',
  'ARXR32B1187FB57099',
  2007,
  209.60608),
 ('SOYMRWW12A6D4FAB14',
  'The Moon And I (Ordinary Day Album Version)',
  'ARKFYS91187B98E58F',
  0,
  267.7024),
 ('SOMJBYD12A6D4F8557',
  'Keepin It Real (Skit)',
  'ARD0S291187B9B7BF5',
  0,
  114.78159),
 ('SOHKNRJ12A6701D1F8', 'Drop of Rain', 'AR10USD1187B99F3F1', 0, 189.57016),
 ('SOIAZJW12AB01853F1', 'Pink World', 'AR8ZCNI1187B9A069B', 1984, 269.81832),
 ('SOUDSGM12AC9618304',
  'Insatiable (Instrumental Version)',
  'ARNTLGG11E2835DDB9',
  0,
  266.39628),
 ('SOQHXMF12AB0182363', 'Young Boy Blues', 'ARGSJW91187B9B1D6B', 0, 218.77506),
 ('SOKEJEJ12A8C13E0D0',

In [29]:
conn.commit()

In [30]:
next_song_logs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6820 entries, 2 to 8055
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   artist         6820 non-null   object 
 1   auth           6820 non-null   object 
 2   firstName      6820 non-null   object 
 3   gender         6820 non-null   object 
 4   itemInSession  6820 non-null   int64  
 5   lastName       6820 non-null   object 
 6   length         6820 non-null   float64
 7   level          6820 non-null   object 
 8   location       6820 non-null   object 
 9   method         6820 non-null   object 
 10  page           6820 non-null   object 
 11  registration   6820 non-null   float64
 12  sessionId      6820 non-null   int64  
 13  song           6820 non-null   object 
 14  status         6820 non-null   int64  
 15  ts             6820 non-null   int64  
 16  userAgent      6820 non-null   object 
 17  userId         6820 non-null   object 
dtypes: float

In [31]:
next_song_logs.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
5,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Quem Quiser Encontrar O Amor,200,1541106496796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
6,The Mars Volta,Logged In,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Eriatarka,200,1541106673796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
7,Infected Mushroom,Logged In,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Becoming Insane,200,1541107053796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


In [32]:
next_song_logs.loc[:,'ts_dt'] = next_song_logs['ts'].apply(
    pd.to_datetime, unit='ms', origin='unix'
)

C:\Users\nicob\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [33]:
pre_songplay_df = next_song_logs[
    ['ts_dt', 'userId', 'level', 'song', 'artist', 'sessionId', 'location', 'userAgent']
]

In [34]:
artists_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69 entries, 0 to 70
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   artist_id  69 non-null     object 
 1   name       69 non-null     object 
 2   location   69 non-null     object 
 3   latitude   31 non-null     float64
 4   longitude  31 non-null     float64
dtypes: float64(2), object(3)
memory usage: 3.2+ KB


In [35]:
first_merge_df = pd.merge(
    left=pre_songplay_df,
    right=artists_df[['artist_id', 'name']],
    how='left',
    left_on='artist',
    right_on='name'
)

In [36]:
songplay_merged_df = pd.merge(
    left=first_merge_df,
    right=songs_df[['song_id', 'title']],
    how='left',
    left_on='song', 
    right_on='title'
)

In [37]:
songplay_merged_df.drop(['name', 'title', 'song', 'artist'], axis=1, inplace=True)
songplay_merged_df

,ts_dt,userId,level,sessionId,location,userAgent,artist_id,song_id
0,2018-11-01 21:01:46.796,8,free,139,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",NaN,NaN
1,2018-11-01 21:05:52.796,8,free,139,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",NaN,NaN
2,2018-11-01 21:08:16.796,8,free,139,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",NaN,NaN
3,2018-11-01 21:11:13.796,8,free,139,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",NaN,NaN
4,2018-11-01 21:17:33.796,8,free,139,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",NaN,NaN
...,...,...,...,...,...,...,...,...
6815,2018-11-30 18:40:05.796,16,paid,1076,"Birmingham-Hoover, AL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",NaN,NaN
6816,2018-11-30 18:44:36.796,16,paid,1076,"Birmingham-Hoover, AL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",NaN,NaN
6817,2018-11-30 18:47:58.796,16,paid,1076,"Birmingham-Hoover, AL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",NaN,NaN
6818,2018-11-30 18:51:24.796,16,paid,1076,"Birmingham-Hoover, AL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",NaN,NaN


In [38]:
correct_order = ['ts_dt', 'userId', 'level', 'song_id', 'artist_id', 'sessionId', 'location', 'userAgent']

songplays_df = songplay_merged_df[correct_order]
songplays_df.rename(columns={
    'ts_dt': 'start_time',
    'userId': 'user_id',
    'sessionId': 'session_id',
    'userAgent': 'user_agent'
}, inplace=True)

In [39]:
songplays_df

,start_time,user_id,level,song_id,artist_id,session_id,location,user_agent
0,2018-11-01 21:01:46.796,8,free,NaN,NaN,139,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK..."
1,2018-11-01 21:05:52.796,8,free,NaN,NaN,139,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK..."
2,2018-11-01 21:08:16.796,8,free,NaN,NaN,139,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK..."
3,2018-11-01 21:11:13.796,8,free,NaN,NaN,139,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK..."
4,2018-11-01 21:17:33.796,8,free,NaN,NaN,139,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK..."
...,...,...,...,...,...,...,...,...
6815,2018-11-30 18:40:05.796,16,paid,NaN,NaN,1076,"Birmingham-Hoover, AL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
6816,2018-11-30 18:44:36.796,16,paid,NaN,NaN,1076,"Birmingham-Hoover, AL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
6817,2018-11-30 18:47:58.796,16,paid,NaN,NaN,1076,"Birmingham-Hoover, AL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
6818,2018-11-30 18:51:24.796,16,paid,NaN,NaN,1076,"Birmingham-Hoover, AL","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."


In [40]:
buffer = StringIO()

songplays_df.to_csv(buffer, header=False, index=False, sep="\t")
buffer.seek(0)

curr.copy_from(buffer, 'songplays', sep="\t", columns=tuple(songplays_df.columns))

In [41]:
conn.commit()

In [42]:
curr.execute("SELECT * FROM artists")
z = pd.DataFrame(curr.fetchall(), columns=['artist_id', 'name', 'location', 'latitude', 'longitude'])